<div style="background-color:	#216375 ; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:black"> Generative AI: Portfolio 1, Diffusion Model from scratch</h1>
<b style="color:white"> Julia Boschman, Daan Eising</b>
    </div>

<a name='start'></a>

## Table of contents

1. [H1: Inleiding](#1.0)
    - [&sect;1.1: Imports](#1.1)
2. [H2: Dataset](#2.0)
    - [&sect;2.1: Let op!](#2.1)
    - [&sect;2.2: Aanmaken nieuwe map](#2.2)
    - [&sect;2.3: Inlezen afbeeldingen bestaande map](#2.3)
    - [&sect;2.4: Weergave selectie dataset](#2.4)
3. [H3: Diffusion Model from scratch](#3.0)
    - [&sect;3.1: Aanmaken parameters Diffusion Model](#3.1)
    - [&sect;3.2: Dataset klaarmaken voor training](#3.2)
    - [&sect;3.3: Het model aanroepen](#3.3)
    - [&sect;3.4: Het model trainen](#3.4)
4. [H4: Vergelijken werkelijke data met gegenereerde data](#4.0)
5. [H5: Korte reflectie op resultaten](#5.0)
6. [H6: Literatuurlijst](#6.0)

[Back to the top](#start)
<a name='1.0'></a>

<div style="background-color:#216375; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H1: Inleiding</strong></h2>
</div>

Voor het eerste portfolio van Generative AI is het de bedoeling om een model uit te kiezen en deze "from scratch" na te bouwen. Dit hebben wij gedaan op basis van deze [paper](https://arxiv.org/pdf/2006.11239) over Diffusion models. Het paper legt in detail uit wat de werking en de wiskunde is achter het model en deelt de resultaten van hun invulling van de Denoising Diffusion Probabilistic Models. In dit notebook laten wij zien hoe het main.py bestandje gebruikt kan worden om een Diffusion model te trainen. Verdere code van het model zelf is dan dus ook te vinden in het main.py bestand.

<a name='1.1'></a>
<h3>&sect;1.1: Imports</h3>

Om er voor te zorgen dat alle modules gedownload kunnen worden doen we eerst een pip install voor alle modules die niet geïnstalleerd zijn.

In [1]:
#!pip install tensorflow
#!pip install tqdm

Nu alle missende modules zijn geïnstalleerd kunnen we de nodige modules importeren. Aangezien alle modules ook worden aangeroepen in "main.py" is het niet nodig om alles in het notebook opnieuw te importeren, maar het geeft een duidelijke weergave, bij het lezen van het notebook, welke modules er allemaal gebruikt worden.

In [ ]:
import os
import numpy as np
from PIL import Image
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from tqdm import tqdm


from main.py import *

[Back to the top](#start)
<a name='2.0'></a>

<div style="background-color:#216375; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H2: Dataset</strong></h2>
</div>

Een Diffusion model kan zowel gebruikt worden voor het genereren van tekst als het genereren van afbeeldingen. De paper over Diffusion Models die wij hebben gelezen gebruikt de toepassing van het genereren van afbeeldingen. Om die reden hebben wij er voor gekozen om in dit notebook ook gebruik te maken van afbeeldingen. 


De afbeeldingen die wij hebben gekozen om te genereren is de [Barok](https://www.kaggle.com/datasets/steubk/wikiart?select=Baroque) dataset van Kaggle's WikiArt. In de dataset staan afbeeldingen van een hoop verschillende soorten kust. Denk hier bijvoorbeeld aan abstract expressionisme, modern realisme, impressionisme, pop art en nog een hoop andere soorten kunst. De reden dat wij hebben gekozen voor de Barok dataset is, omdat het ons gaaf leek om nieuwe kunst te genereren van een stijl die nog niet vaak wordt gebruikt in de generatieve AI. Hierdoor zagen wij het als een leuke uitdaging en zal het leuk zijn om een resultaat te zien van een nieuw kunstwerk met een stijl uit de zeventiende en achttiende eeuw.


Om dit notebook zelf te kunnen runnen is het belangrijk om eerst de boven genoemde dataset te downloaden van Kaggle. Dit is gemakkelijk en gratis te doen met een Kaggle account. Hierdoor is het ook mogelijk voor andere mensen om gebruik te maken van onze toepassing.

<a name='2.1'></a>
<h3>&sect;2.1: Let op</h3>

### Let op: Er staan hieronder twee cellen code waarvan je er maar eentje hoeft te runnen. Het verschil zit er in of je de afbeeldingen al hebt ge-resized of niet. Kijk dus even goed welke je moet pakken. Daarnaast wordt er meerdere keren dataset_128 gebruikt, omdat ik de kleinere dimensie gebruik. Als je andere data gebruikt, pas de plekken waar dataset_128 staat dan aan naar de naam die jij de afbeeldingen hebt gegeven.

<a name='2.2'></a>
<h3>&sect;2.2: Aanmaken nieuwe map</h3>

Voor het aanmaken van een nieuwe map, gebruik deze code!

In [ ]:
IMAGE_DIR = "Baroque"
SAVE_DIR = "Processed_Baroque_32"

# Grootte van de afbeeldingen
IMG_SIZE = (32, 32)
os.makedirs(SAVE_DIR, exist_ok=True)

dataset_32 = preprocess_and_save_images(IMAGE_DIR, SAVE_DIR, IMG_SIZE)
print(f"Dataset geladen met vorm: {dataset_32.shape}")

<a name='2.3'></a>
<h3>&sect;2.3: Inlezen afbeeldingen bestaande map</h3>

Voor het inlezen van de afbeeldingen uit een bestaande map, gebruik onderstaande code!

In [ ]:
dataset_32 = load_images_from_folder("Processed_Baroque_32")

<a name='2.4'></a>
<h3>&sect;2.4: Weergave selectie dataset</h3>

Voor het laten zien van een enkeling van de foto's.

In [ ]:
show_images(dataset_32, num_images=9)

[Back to the top](#start)
<a name='3.0'></a>

<div style="background-color:#216375; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H3: Diffusion Model from scratch</strong></h2>
</div>

<a name='3.1'></a>
<h3>&sect;3.1: Aanmaken parameters Diffusion Model</h3>

In [ ]:
T = 250
beta = np.linspace(1e-4, 0.02, T)
alpha = 1 - beta
alpha_bar = np.cumprod(alpha)

<a name='3.2'></a>
<h3>&sect;3.2: Dataset klaarmaken voor training</h3>

In [ ]:
data = dataset_32.reshape(-1, 32*32*3)  # Flatten de afbeeldingen naar vectoren
data = (data / 127.5) - 1.0  # Normaliseer naar [-1,1]

<a name='3.3'></a>
<h3>&sect;3.3: Het model aanroepen</h3>

In [ ]:
model = DiffusionModel(input_dim=32*32*3, t_embed_dim=16, hidden_dims=[512, 1024, 512], output_dim=32*32*3)

<a name='3.4'></a>
<h3>&sect;3.4: Het model trainen</h3>

In [ ]:
train_diffusion_model(model, data, epochs=200, batch_size=16)

[Back to the top](#start)
<a name='4.0'></a>

<div style="background-color:#216375; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H4: Vergelijken werkelijke data met gegenereerde data</strong></h2>
</div>

Na het trainen van het model wordt het tijd om te kijken naar de prestatie van dit model. Zo vergelijken wij een set aan de afbeeldingen uit de Barok dataset met een setje gegenereerde data, om te kijken of deze enigszinds met elkaar overeenkomen.

In [ ]:
generated_samples = sample(model, shape=(5, 32, 32, 3))
compare_generated_vs_real(generated_samples, dataset_128[:5])

[Back to the top](#start)
<a name='5.0'></a>

<div style="background-color:#216375; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H5: Korte reflectie op resultaten</strong></h2>
</div>

Helaas is het ons niet gelukt om een afbeelding te genereren die overeenkomt met de werkelijke data. In plaats daarvan genereren wij afbeeldingen met ruis. Het zou kunnen dat ons model minder goed presteert door het hebben van te weinig data, het gebruiken van te weinig lagen of het niet gebruiken van genoeg epochs tijdens het trainen.


In de tijd die wij hadden, hebben wij veel tweaks moeten doen in onze code van het model. Zo hebben wij meer lagen toegevoegd, gebruik gemaakt van een andere activatiefunctie en het aantal epochs tijdens het trainen verhoogd. Jammer genoeg hebben wij beide geen beschikking tot een computer met GPU, waardoor het steeds lastiger werd om aanpassingen te maken in de code die er niet voor zorgde dat de training extreem lang zou duren (3 uur per epoch). Wel hebben wij een trainingsronde behaald waar de loss bleef dalen, maar uiteindelijk vast kwam hangen in een loss van 0.86. Voor een goed model zouden de waardes van de loss tussen de 0.1 en 0.01 moeten liggen. Dit hebben wij echter niet voor elkaar gekregen.

[Back to the top](#start)
<a name='6.0'></a>

<div style="background-color:#216375; text-align:center; vertical-align:middle; padding:10px 0; margin-top:5px; margin-bottom:5px">
        <h2 style="color:white"><strong>H6: Literatuurlijst</strong></h2>
</div>

- [Chat GPT voor verbeteren van diffusion](https://chatgpt.com/c/67d9c8f9-6410-8000-94a6-6dc0743076da)
- [Git met Diffusion Model](https://github.com/dome272/Diffusion-Models-pytorch/blob/main/ddpm.py)